### Objective

In this notebook, we develop a global optimization routine to find the design that minimizes the weight while satisfying the geometrical and thermal constraints.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from deap import base, creator, tools, algorithms

from two_sources import thermal_distribution_maxT

### 1. Setup constants

In [2]:
Q1, Q2 = 300, 200

In [3]:
Data = (25, 50e-3, 65e-3, 61.4e-3, 106e-3)
Tjmax = 175
c_module, d_module = 61.4e-3, 106e-3

d_min, d_max = 5e-3, 30e-3
b_min, b_max = 73.7e-3, 307e-3
L_min, L_max = 127.2e-3, 530e-3
c_min, c_max = 10e-3, 39e-3
L_duct_min, L_duct_max = 20e-3, 50e-3
n_min, n_max = 10, 50
t_min, t_max = 1e-3, b_max/n_min - 1e-3
Xc1_min, Xc1_max = c_module / 2, b_max - c_module / 2
Xc2_min, Xc2_max = c_module / 2, b_max - c_module / 2
Yc1_min, Yc1_max = d_module / 2, L_max - d_module / 2
Yc2_min, Yc2_max = d_module / 2, L_max - d_module / 2

#### 2. Problem definition

In [4]:
def objective(x):

    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    # Calculate weight
    density_Al = 2700
    Fan_height = 40e-3
    Fan_Weight = 50.8e-3
    N_fan = np.ceil(b / Fan_height)

    # Weight calculation
    w = density_Al*(b*d*L+n*(c*t*L))+ Fan_Weight*N_fan
    
    return w

In [5]:
def constraint_maxT(x, Q1, Q2, Data, Tmax_threshold):    
    
    # Calculate Tmax
    input = np.hstack((Q1, Q2, x))
    Tmax, _ = thermal_distribution_maxT(input, Data)
    
    # Constraint Tmax to be less than or equal to Tmax_threshold
    if Tmax_threshold >= Tmax:
        return 0
    else:
        return 10000

In [6]:
def non_overlap_constraint(x, c_module, d_module):

    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x
    
    # Calculate distances
    xc_dist = abs(xc1 - xc2)  # Assuming these are Xc_first and Xc_second
    yc_dist = abs(yc1 - yc2)  # Assuming these are Yc_first and Yc_second
    
    # Constraints for non-overlapping
    if max(xc_dist - c_module, yc_dist - d_module) >= 0:
        return 0
    else:
        return 10000

In [7]:
def t_upper_bound(x):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    if b / n - 1e-3 - t >= 0:
        return 0
    else:
        return 10000

In [8]:
def x1_upper_bound(x, c_module, d_module):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    Xc_max = b - c_module / 2

    if Xc_max - xc1 >= 0:
        return 0
    else:
        return 10000

In [9]:
def x1_lower_bound(x, c_module, d_module):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    Xc_min = c_module / 2

    if xc1 - Xc_min >= 0:
        return 0
    else:
        return 10000

In [10]:
def x2_upper_bound(x, c_module, d_module):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    Xc_max = b - c_module / 2

    if Xc_max - xc2 >= 0:
        return 0
    else:
        return 10000

In [11]:
def x2_lower_bound(x, c_module, d_module):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    Xc_min = c_module / 2

    if xc2 - Xc_min >= 0:
        return 0
    else:
        return 10000

In [12]:
def y1_upper_bound(x, c_module, d_module):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    Yc_max = L - d_module / 2

    if Yc_max - yc1 >= 0:
        return 0
    else:
        return 10000

In [13]:
def y1_lower_bound(x, c_module, d_module):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    Yc_min = d_module / 2

    if yc1 - Yc_min >= 0:
        return 0
    else:
        return 10000

In [14]:
def y2_upper_bound(x, c_module, d_module):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    Yc_max = L - d_module / 2

    if Yc_max - yc2 >= 0:
        return 0
    else:
        return 10000

In [15]:
def y2_lower_bound(x, c_module, d_module):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    Yc_min = d_module / 2

    if yc2 - Yc_min >= 0:
        return 0
    else:
        return 10000

In [16]:
def make_eval_function(Q1, Q2, Data, c_module, d_module, Tmax_threshold, n_min, n_max):
    def eval_individual(individual):
        # Objective function value
        new_n = int(individual[5])
        individual[5] = max(min(new_n, n_max), n_min)
        objective_value = objective(individual)

        # Penalty for constraint violations
        if objective_value > 0:
            penalty = 0
        else:
            penalty = 10000
            
        try:
            penalty_t = constraint_maxT(individual, Q1, Q2, Data, Tmax_threshold)
        except Exception as e:
            penalty_t = 10000
            
        penalty += penalty_t
        penalty += non_overlap_constraint(individual, c_module, d_module)
        penalty += t_upper_bound(individual)
        penalty += x1_upper_bound(individual, c_module, d_module)
        penalty += x1_lower_bound(individual, c_module, d_module)
        penalty += x2_upper_bound(individual, c_module, d_module)
        penalty += x2_lower_bound(individual, c_module, d_module)
        penalty += y1_upper_bound(individual, c_module, d_module)
        penalty += y1_lower_bound(individual, c_module, d_module)
        penalty += y2_upper_bound(individual, c_module, d_module)
        penalty += y2_lower_bound(individual, c_module, d_module)

        return objective_value + penalty, 

    return eval_individual

In [17]:
def custom_mutate(individual, low, up, eta, indpb):
    # Clone the individual to avoid altering the original during the iteration
    # Mutate using mutPolynomialBounded for continuous variables
    _, = tools.mutPolynomialBounded(individual, low, up, eta, indpb)
    
    # Copy back the mutated values except for 'n' at index 5
    individual[5] = int(round(individual[5]))

    return individual,

In [18]:
# Minimization problem. Adjust weights for maximization.
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

# Initialize DEAP tools for GA
toolbox = base.Toolbox()

# Individual and population setup
# Adjust attribute generators according to your problem's variables
toolbox.register("individual", tools.initCycle, creator.Individual, 
                 (lambda: random.uniform(d_min, d_max),
                  lambda: random.uniform(b_min, b_max),
                  lambda: random.uniform(L_min, L_max),
                  lambda: random.uniform(c_min, c_max),
                  lambda: random.uniform(L_duct_min, L_duct_max),
                  lambda: random.randint(n_min, n_max),
                  lambda: random.uniform(t_min, t_max),
                  lambda: random.uniform(Xc1_min, Xc1_max),
                  lambda: random.uniform(Yc1_min, Yc1_max),
                  lambda: random.uniform(Xc2_min, Xc2_max),
                  lambda: random.uniform(Yc2_min, Yc2_max)), n=1)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Register genetic operators
toolbox.register("evaluate", make_eval_function(Q1, Q2, Data, c_module, d_module, Tjmax, n_min, n_max))
toolbox.register("mate", tools.cxBlend, alpha=0.5)
# Update mutation registration to use the custom mutation function
toolbox.register("mutate", tools.mutPolynomialBounded, 
                 low=[d_min, b_min, L_min, c_min, L_duct_min, n_min, t_min, Xc1_min, Yc1_min, Xc2_min, Yc2_min], 
                 up=[d_max, b_max, L_max, c_max, L_duct_max, n_max, t_max, Xc1_max, Yc1_max, Xc2_max, Yc2_max], 
                 eta=1.0, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Genetic algorithm parameters
population_size = 10
crossover_probability = 0.7
mutation_probability = 0.5
number_of_generations = 200

# Initialize population
pop = toolbox.population(n=population_size)

# Statistics accumulator
stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)
stats.register("max", np.max)

In [19]:
%%time

hof_size = 1
hall_of_fame = tools.HallOfFame(hof_size)

# Evolutionary algorithm execution
final_pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=crossover_probability, 
                                   mutpb=mutation_probability,
                                   ngen=number_of_generations, stats=stats, 
                                   halloffame=hall_of_fame, verbose=True)

D:\PE-Optimization\new_campaign\two_sources.py:67: RuntimeWarning: invalid value encountered in sqrt
  fRe_fd = 12 / (np.sqrt(EPS) * (1 + EPS) * (1 - 192 * EPS * np.tanh(np.pi / 2 / EPS) / np.pi ** 5))
D:\PE-Optimization\new_campaign\two_sources.py:69: RuntimeWarning: invalid value encountered in sqrt
  fapp = n * visc_air_K * np.sqrt(c * s) * fRe / V


gen	nevals	avg    	min    	max  
0  	10    	41013.5	20018.8	60009
1  	10    	39018.5	10007.5	70006.9
2  	7     	28021.9	10007.5	40039.6
3  	8     	16013.4	7.56861	40015.9
4  	7     	9009.25	7.12723	30024.5
5  	10    	6006.9 	7.12723	10006.8
6  	7     	4008.88	3.94849	10013.1


D:\PE-Optimization\new_campaign\two_sources.py:104: RuntimeWarning: invalid value encountered in sqrt
  eff_fin = np.tanh(np.sqrt(2 * h * (t + L) / lambda_HS / t / L) * c) / np.sqrt(2 * h * (t + L) / lambda_HS / t / L) / c


7  	7     	9009.12	3.94849	20019.5
8  	9     	13010.8	7.12723	30013.2
9  	10    	5010.34	7.45676	20012.4
10 	10    	5007.99	6.42066	30003.9
11 	9     	1008.46	6.42066	10006.4
12 	9     	5010.07	6.25457	30020.3
13 	9     	2007.63	6.25457	10009.8
14 	7     	7006.82	6.25457	30005.2
15 	7     	3008.47	6.25457	20018.1
16 	10    	5006.86	5.58161	20002.5
17 	9     	7007.94	6.32992	20006.6
18 	9     	1010.67	5.66867	10007.9
19 	7     	2007.34	5.66867	10007.3
20 	10    	6.77941	4.86614	8.95643
21 	4     	6.03168	4.20335	7.46488
22 	7     	1005.32	4.20335	10002.7
23 	8     	5.29437	4.13335	10.8792
24 	9     	2005.01	4.08811	10004.7
25 	7     	2004.5 	4.08811	10004.3
26 	10    	4.55114	3.97702	6.27112
27 	9     	1004.21	3.70583	10004  
28 	10    	3003.91	3.52249	10004.4
29 	10    	1004.4 	3.50683	10004.3
30 	8     	8004.92	3.50683	30015.9
31 	7     	3.73507	3.49179	4.11547
32 	9     	4004.44	3.49179	20002.1
33 	9     	2004.3 	3.49179	10003.7
34 	9     	2003.58	3.46972	20001.3
35 	9     	4003.45	2

In [20]:
best_ind = hall_of_fame[0]
print(f"Best fitness:", best_ind.fitness.values[0])
columns = ['d', 'b', 'L', 'c', 'L_duct', 'n', 't', 'xc1', 'yc1', 'xc2', 'yc2']
df = pd.DataFrame({'optimized': best_ind})
df.index = columns
df

Best fitness: 0.8469661448825472


,optimized
d,0.003743
b,0.233408
L,0.209702
c,0.008382
L_duct,0.020501
n,10.000000
t,0.001001
xc1,0.084365
yc1,0.098307
xc2,0.181150


In [21]:
# First Generation
best_ind = tools.selBest(final_pop, 1)[0]
print(f"Best fitness:", best_ind.fitness.values[0])
columns = ['d', 'b', 'L', 'c', 'L_duct', 'n', 't', 'xc1', 'yc1', 'xc2', 'yc2']
df = pd.DataFrame({'optimized': best_ind})
df.index = columns
df

Best fitness: 0.9920846027709415


,optimized
d,0.004993
b,0.224799
L,0.208038
c,0.010113
L_duct,0.013895
n,10.000000
t,0.001001
xc1,0.060799
yc1,0.065122
xc2,0.192268


#### Verification

In [22]:
def verification_suite(design, Q1, Q2, Data, Tjmax, c_module, d_module):
    
    # Tmax constraint
    check = constraint_maxT(design, Q1, Q2, Data, Tjmax)
    if check >= 0:
        print(f"Tmax verification: True")
    else:
        print(f"Tmax exceeded by {check}")
    
    # Coordinates bounds
    check = x1_upper_bound(design, c_module, d_module)
    if check >= 0:
        print(f"x1 upper bound: True")
    else:
        print(f"x1 upper bound violated: {check}")

    check = x1_lower_bound(design, c_module, d_module)
    if check >= 0:
        print(f"x1 lower bound: True")
    else:
        print(f"x1 lower bound violated: {check}")

    check = x2_upper_bound(design, c_module, d_module)
    if check >= 0:
        print(f"x2 upper bound: True")
    else:
        print(f"x2 upper bound violated: {check}")

    check = x2_lower_bound(design, c_module, d_module)
    if check >= 0:
        print(f"x2 lower bound: True")
    else:
        print(f"x2 lower bound violated: {check}")

    check = y1_upper_bound(design, c_module, d_module)
    if check >= 0:
        print(f"y1 upper bound: True")
    else:
        print(f"y1 upper bound violated: {check}")

    check = y1_lower_bound(design, c_module, d_module)
    if check >= 0:
        print(f"y1 lower bound: True")
    else:
        print(f"y1 lower bound violated: {check}")

    check = y2_upper_bound(design, c_module, d_module)
    if check >= 0:
        print(f"y2 upper bound: True")
    else:
        print(f"y2 upper bound violated: {check}")

    check = y2_lower_bound(design, c_module, d_module)
    if check >= 0:
        print(f"y2 lower bound: True")
    else:
        print(f"y2 lower bound violated: {check}")

    
    # Non-overlapping bounds
    check = non_overlap_constraint(design, c_module, d_module)
    if check >= 0:
        print(f"Non-overlapping True")
    else:
        print(f"Non-overlapping violated: {check}")
    
    # t bounds
    check = t_upper_bound(design)
    if check >= 0:
        print(f"t bound verification True")
    else:
        print(f"t bound violated: {check}")

In [23]:
verification_suite(best_ind, Q1, Q2, Data, Tjmax, c_module, d_module)

Tmax verification: True
x1 upper bound: True
x1 lower bound: True
x2 upper bound: True
x2 lower bound: True
y1 upper bound: True
y1 lower bound: True
y2 upper bound: True
y2 lower bound: True
Non-overlapping True
t bound verification True
